# Задача потребителя

## Получение данных

Параметры модулирования:


>  Станция
    Novosibirsk(MRK)
    
    X
    452.28865
    Y
    3635.913158
    Z
    5203.397861

>  Спутники

*   1. 1/2
*   2. 2/1
*   3. 3/12
*   4. 4/6


In [0]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

In [0]:
 required_coordinates = np.array([
        452.28865,
        3635.913158,
        5203.397861,
 ])

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('Загруженный "{name}" размером {length} байт'.format(
      name=fn, length=len(uploaded[fn])))

Saving reports.xls to reports.xls
Загруженный "reports.xls" размером 306688 байт


In [4]:
mod_data = pd.read_excel("reports.xls")
mod_data.head()

,Time_mom,N_Sat,P1,Sat_X,Sat_Y,Sat_Z
0,01:47:00,1,23640.463423,-1403.143993,25077.821790,-4577.880104
1,01:47:30,1,23614.250832,-1400.494773,25096.862175,-4472.742878
2,01:48:00,1,23588.052424,-1397.739724,25115.469797,-4367.509327
3,01:48:30,1,23561.868740,-1394.880788,25133.643881,-4262.181717
4,01:49:00,1,23535.700307,-1391.919910,25151.383669,-4156.762322


In [5]:
by_time = mod_data.groupby(by='Time_mom')['N_Sat'].count() 
observed_more_than_two = by_time[by_time>2]
len(observed_more_than_two)

405

In [6]:
len(observed_more_than_two[observed_more_than_two>3])   # одновременно 4 никогда не наблюдаются

0

In [7]:
sample_time = observed_more_than_two.index[78]
sample = mod_data[mod_data['Time_mom'] == sample_time]
sample

,Time_mom,N_Sat,P1,Sat_X,Sat_Y,Sat_Z
785,05:25:00,2,19052.577323,2184.905275,15349.310978,20129.689073
786,05:25:00,1,21106.787484,-11915.881636,3162.669722,22300.104699
787,05:25:00,3,21880.141257,16211.479975,18786.485990,6124.669437


## Предустановки

In [0]:
def get_Us_and_P(row):
    return np.array([row['Sat_X'],
                     row['Sat_Y'],
                     row['Sat_Z'],]), row['P1']

In [0]:
def distance(v1,v2):
    return np.sqrt(np.sum((v1-v2)**2))

In [0]:
def dir_cos(Us,Ur,Ro0):
    return (Us-Ur)/Ro0

## I способ расчета

In [11]:
sat_data = [get_Us_and_P(sample.iloc[i]) for i in range(3)]
sat_data

[(array([ 2184.90527483, 15349.3109776 , 20129.68907288]), 19052.5773226953),
 (array([-11915.8816357 ,   3162.66972196,  22300.10469944]), 21106.787484125),
 (array([16211.47997461, 18786.48599038,  6124.66943681]), 21880.1412569171)]

In [0]:
def to_solve_Ur_0(v,Sat_data):
    return [distance(v,u) - p for u,p in Sat_data]

In [13]:
to_solve_Ur_0(required_coordinates,sat_data)

[-0.02321069562822231, -0.0828553766150435, 0.002930710372311296]

In [14]:
%time Ur_0 = fsolve(to_solve_Ur_0, np.array([1.,1.,1.]), args=(sat_data))

Ur_0              

CPU times: user 993 µs, sys: 0 ns, total: 993 µs
Wall time: 988 µs


array([ 450.6806568 , 3637.66420946, 5202.18098915])

In [15]:
Ro = np.array([P_i for Us_i,P_i in sat_data])
Ro_0 = Ro.round()
Us = [Us_i for Us_i,P in sat_data]
Us

[array([ 2184.90527483, 15349.3109776 , 20129.68907288]),
 array([-11915.8816357 ,   3162.66972196,  22300.10469944]),
 array([16211.47997461, 18786.48599038,  6124.66943681])]

In [16]:
dir_cosines = [dir_cos(*vals) for vals in zip(Us,[Ur_0]*3,Ro_0)]
dir_cosines

[array([0.09102108, 0.61468781, 0.78347284]),
 array([-0.58589863, -0.02250412,  0.8100594 ]),
 array([0.72032904, 0.69235931, 0.04216126])]

In [0]:
def to_solve_deltas(deltas,cosines,ro_0,ro):
    return [(-p+p0+np.sum(np.array(deltas)*cos)) for cos,p0,p in zip(cosines,ro_0,ro)]

In [29]:
to_solve_deltas([.0,.0,.0,],dir_cosines,Ro_0,Ro)

[0.42267730470121023, 0.212515874998644, -0.14125691710069077]

In [30]:
%time deltas = fsolve(to_solve_deltas,[.0,.0,.0],args=(dir_cosines,Ro_0,Ro))
deltas

CPU times: user 0 ns, sys: 1.96 ms, total: 1.96 ms
Wall time: 4.82 ms


array([ 11.38832422, -12.1094908 ,   7.63817345])

In [21]:
print('Без смещений')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0)
print('разности в координатах\n', abs(required_coordinates - Ur_0))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0))

Без смещений
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 450.6806568  3637.66420946 5202.18098915]
разности в координатах
 [1.6079932  1.75105146 1.21687185]
расстояние между реальной и предполагаемой точками
 2.6706928744430445


In [34]:
print('Со смещениями + delta')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0+deltas)
print('разности в координатах\n', abs(required_coordinates - Ur_0-deltas))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0+deltas))

Со смещениями
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 462.06898102 3625.55471865 5209.8191626 ]
разности в координатах
 [ 9.78033102 10.35843935  6.4213016 ]
расстояние между реальной и предполагаемой точками
 15.626428087904301


In [0]:
print('Со смещениями - delta')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0-deltas)
print('разности в координатах\n', abs(required_coordinates - Ur_0+deltas))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0-deltas))

Со смещениями
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 462.06898102 3625.55471865 5209.8191626 ]
разности в координатах
 [ 9.78033102 10.35843935  6.4213016 ]
расстояние между реальной и предполагаемой точками
 15.626428087904301


## II способ расчета

In [45]:
print(sat_data)
print(Us)
print(Ro)

[(array([ 2184.90527483, 15349.3109776 , 20129.68907288]), 19052.5773226953), (array([-11915.8816357 ,   3162.66972196,  22300.10469944]), 21106.787484125), (array([16211.47997461, 18786.48599038,  6124.66943681]), 21880.1412569171)]
[array([ 2184.90527483, 15349.3109776 , 20129.68907288]), array([-11915.8816357 ,   3162.66972196,  22300.10469944]), array([16211.47997461, 18786.48599038,  6124.66943681])]
[19052.5773227  21106.78748413 21880.14125692]


In [41]:
%time Ur_0 = fsolve(to_solve_Ur_0, np.array([1.,1.,1.]), args=(sat_data)).round()
Ur_0              

CPU times: user 902 µs, sys: 0 ns, total: 902 µs
Wall time: 879 µs


array([ 451., 3638., 5202.])

In [47]:
Ro_0 = np.array([distance(Ur_0,Us_i) for Us_i in Us])
Ro_0

array([19052.48365365, 21107.12876204, 21879.68637162])

In [48]:
dir_cosines = [dir_cos(*vals) for vals in zip(Us,[Ur_0]*3,Ro_0)]
dir_cosines

[array([0.09100678, 0.61468684, 0.78350358]),
 array([-0.58591018, -0.02251989,  0.81006303]),
 array([0.72032477, 0.69235389, 0.04217014])]

In [49]:
%time deltas = fsolve(to_solve_deltas,[.0,.0,.0],args=(dir_cosines,Ro_0,Ro))
deltas

CPU times: user 1.54 ms, sys: 0 ns, total: 1.54 ms
Wall time: 2.19 ms


array([ 0.31926152,  0.33588008, -0.18104191])

In [50]:
print('Без смещений')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0)
print('разности в координатах\n', abs(required_coordinates - Ur_0))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0))

Без смещений
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 451. 3638. 5202.]
разности в координатах
 [1.28865  2.086842 1.397861]
расстояние между реальной и предполагаемой точками
 2.8230380321183715


In [53]:
print('Со смещениями - delta')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0-deltas)
print('разности в координатах\n', abs(required_coordinates - Ur_0+deltas))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0-deltas))

Со смещениями - delta
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 450.68073848 3637.66411992 5202.18104191]
разности в координатах
 [1.60791152 1.75096192 1.21681909]
расстояние между реальной и предполагаемой точками
 2.6705609482341104


In [54]:
print('Со смещениями + delta')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0+deltas)
print('разности в координатах\n', abs(required_coordinates - Ur_0-deltas))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0+deltas))

Со смещениями + delta
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 451.31926152 3638.33588008 5201.81895809]
разности в координатах
 [0.96938848 2.42272208 1.57890291]
расстояние между реальной и предполагаемой точками
 3.0499558559262883


## III способ расчета

In [0]:
Ro_0 = Ro.round()

In [59]:
%time Ur_0 = fsolve(to_solve_Ur_0, np.array([1.,1.,1.]), args=(list(zip(Us,Ro_0))))
Ur_0              

CPU times: user 889 µs, sys: 0 ns, total: 889 µs
Wall time: 886 µs


array([ 462.25086458, 3625.36858019, 5209.95564882])

In [60]:
dir_cosines = [dir_cos(*vals) for vals in zip(Us,[Ur_0]*3,Ro_0)]
dir_cosines

[array([0.09041381, 0.61533314, 0.78306479]),
 array([-0.58644679, -0.02192158,  0.80969105]),
 array([0.71980023, 0.69292127, 0.04180593])]

In [61]:
%time deltas = fsolve(to_solve_deltas,[.0,.0,.0],args=(dir_cosines,Ro_0,Ro))
deltas

CPU times: user 1.49 ms, sys: 52 µs, total: 1.54 ms
Wall time: 1.34 ms


array([ 11.75824137, -12.48807544,   7.91574599])

In [62]:
print('Без смещений')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0)
print('разности в координатах\n', abs(required_coordinates - Ur_0))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0))

Без смещений
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 462.25086458 3625.36858019 5209.95564882]
разности в координатах
 [ 9.96221458 10.54457781  6.55778782]
расстояние между реальной и предполагаемой точками
 15.919749418286411


In [63]:
print('Со смещениями - delta')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0-deltas)
print('разности в координатах\n', abs(required_coordinates - Ur_0+deltas))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0-deltas))

Со смещениями - delta
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 450.4926232  3637.85665563 5202.03990283]
разности в координатах
 [1.7960268  1.94349763 1.35795817]
расстояние между реальной и предполагаемой точками
 2.9743815625166876


In [64]:
print('Со смещениями + delta')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0+deltas)
print('разности в координатах\n', abs(required_coordinates - Ur_0-deltas))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0+deltas))

Со смещениями + delta
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 474.00910595 3612.88050475 5217.87139481]
разности в координатах
 [21.72045595 23.03265325 14.47353381]
расстояние между реальной и предполагаемой точками
 34.810407972929994


## Результаты

In [66]:
print('Δ','δ')

Δ δ


In [0]:
def ingot():
    return    {'Xr без коррекции':[],
               'Yr без коррекции':[],
               'Zr без коррекции':[],
               'ΔX':[],
               'ΔY':[],
               'ΔZ':[],}

In [0]:
def method_1(samples: list):
    res = ingot()
    for sample in samples:
        sat_data = [get_Us_and_P(sample.iloc[i]) for i in range(3)]
        Ur_0 = fsolve(to_solve_Ur_0, np.array([1.,1.,1.]), args=(sat_data))
        Ro = np.array([P_i for Us_i,P_i in sat_data])
        Ro_0 = Ro.round()
        Us = [Us_i for Us_i,P in sat_data]
        dir_cosines = [dir_cos(*vals) for vals in zip(Us,[Ur_0]*3,Ro_0)]
        deltas = fsolve(to_solve_deltas,[.0,.0,.0],args=(dir_cosines,Ro_0,Ro))
        for i,ax in enumerate(('X','Y','Z')):
           res[ax+'r без коррекции'].append(Ur_0[i])
           res['Δ'+ax].append(deltas[i])
    return res

In [0]:
def method_2(samples: list):
    res = ingot()
    for sample in samples:
        sat_data = [get_Us_and_P(sample.iloc[i]) for i in range(3)]
        Ro = np.array([P_i for Us_i,P_i in sat_data])
        Us = [Us_i for Us_i,P in sat_data]
        Ur_0 = fsolve(to_solve_Ur_0, np.array([1.,1.,1.]), args=(sat_data)).round()
        Ro_0 = np.array([distance(Ur_0,Us_i) for Us_i in Us])
        dir_cosines = [dir_cos(*vals) for vals in zip(Us,[Ur_0]*3,Ro_0)]
        deltas = fsolve(to_solve_deltas,[.0,.0,.0],args=(dir_cosines,Ro_0,Ro))
        for i,ax in enumerate(('X','Y','Z')):
           res[ax+'r без коррекции'].append(Ur_0[i])
           res['Δ'+ax].append(deltas[i])
    return res

In [0]:
def method_3(samples: list):
    res = ingot()
    for sample in samples:
        sat_data = [get_Us_and_P(sample.iloc[i]) for i in range(3)]
        Ro = np.array([P_i for Us_i,P_i in sat_data])
        Us = [Us_i for Us_i,P in sat_data]
        Ro_0 = Ro.round()
        Ur_0 = fsolve(to_solve_Ur_0, np.array([1.,1.,1.]), args=(list(zip(Us,Ro_0))))
        dir_cosines = [dir_cos(*vals) for vals in zip(Us,[Ur_0]*3,Ro_0)]
        deltas = fsolve(to_solve_deltas,[.0,.0,.0],args=(dir_cosines,Ro_0,Ro))
        for i,ax in enumerate(('X','Y','Z')):
           res[ax+'r без коррекции'].append(Ur_0[i])
           res['Δ'+ax].append(deltas[i])
    return res

In [0]:
from random import sample as some_samples

In [72]:
n_sample_time = some_samples(observed_more_than_two.index.to_list(), 12)
samples = [mod_data[mod_data['Time_mom'] == sample_time] for sample_time in n_sample_time]
samples[4:8]

[      Time_mom  N_Sat            P1         Sat_X         Sat_Y         Sat_Z
 1665  07:52:00      2  22797.102214 -20224.854103   4282.385819  14781.527912
 1666  07:52:00      3  19414.309175  -2591.632905  10724.703544  23019.050184
 1667  07:52:00      4  20258.056522  14697.783003  10829.685964  17681.547844,
       Time_mom  N_Sat            P1         Sat_X         Sat_Y         Sat_Z
 1079  06:14:00      2  19480.337107  -4734.848496   9726.494539  22965.154933
 1080  06:14:00      1  23166.568085 -18509.512818  -1619.420214  17431.064071
 1081  06:14:00      3  20186.330641  12930.758302  15907.933548  15261.883423,
       Time_mom  N_Sat            P1         Sat_X         Sat_Y         Sat_Z
 1599  07:41:00      2  22337.574802 -18794.335354   4423.674440  16513.105573
 1600  07:41:00      3  19317.962686   -529.011020  10979.680311  23044.039060
 1601  07:41:00      4  20577.455360  16201.444918  11194.953370  16077.981843,
       Time_mom  N_Sat            P1         Sat_

In [0]:
results = [pd.DataFrame(eval('method_'+str(i+1)+'(samples)')) for i in range(3)]

for df in results:
    to_pop = list()
    for sign in ('+','-'):
        for i,ax in enumerate(('X','Y','Z')):
            tmp = eval("df[ax+'r без коррекции']"+sign+
                       "df['Δ'+ax] - required_coordinates[i]" )
            df['{a}r {s} Δ{a} - {a} реал.'.format(a=ax,s=sign)] = tmp
            hs = str(i)+sign
            to_pop.append(hs)
            df[hs] = tmp**2
        df['Расстояние от реал до (Ur{s}Δ)'.format(s=sign)] = np.sqrt(
            df['0'+sign]+df['1'+sign]+df['2'+sign]
        )
    for hs in to_pop:
        df.pop(hs)
    to_pop.clear()

In [92]:
results[0]

,Xr без коррекции,Yr без коррекции,Zr без коррекции,ΔX,ΔY,ΔZ,Xr + ΔX - X реал.,Yr + ΔY - Y реал.,Zr + ΔZ - Z реал.,Расстояние от реал до (Ur+Δ),Xr - ΔX - X реал.,Yr - ΔY - Y реал.,Zr - ΔZ - Z реал.,Расстояние от реал до (Ur-Δ)
0,217.905615,4317.027841,4927.659635,6.661148,-19.501511,7.876834,-227.721887,661.613172,-267.861392,749.225582,-241.044183,700.616193,-283.615060,793.349136
1,450.897530,3637.994006,5202.191821,13.920197,-19.870024,10.895871,12.529077,-17.789176,9.689831,23.818593,-15.311317,21.950871,-12.101910,29.372324
2,450.773399,3637.680450,5202.217625,-22.198653,25.134624,-16.207920,-23.713905,26.901917,-17.388157,39.854866,20.683402,-23.367332,15.027684,34.636204
3,453.076604,3634.098063,5204.159172,7.058420,-17.120661,8.014128,7.846374,-18.935755,8.775439,22.296563,-6.270466,15.305566,-7.252818,18.060522
4,451.932368,3638.737791,5202.170531,-1.238149,7.729179,-2.948595,-1.594430,10.553812,-4.175924,11.461391,0.881867,-4.904547,1.721265,5.272098
5,450.849867,3638.482798,5202.042048,-10.114834,18.011806,-8.756445,-11.553617,20.581446,-10.112257,25.677611,8.676050,-15.442166,7.400632,19.196450
6,451.528518,3640.064742,5201.610193,1.718565,-5.935167,2.495520,0.958433,-1.783583,0.707852,2.144951,-2.478698,10.086751,-4.283188,11.235310
7,450.880169,3638.204723,5202.129578,9.869075,-14.712549,7.830190,8.460593,-12.420984,6.561907,16.398814,-11.277556,17.004114,-9.098473,22.340666
8,454.147681,3630.270821,5205.766351,-1.962010,4.920221,-2.167863,-0.102978,-0.722116,0.200627,0.756510,3.821041,-10.562558,4.536353,12.113897
9,453.955781,3630.961902,5205.474211,-4.073163,12.443391,-5.653850,-2.406032,7.492135,-3.577500,8.644049,5.740295,-17.394648,7.730201,19.881669


In [93]:
results[1]

,Xr без коррекции,Yr без коррекции,Zr без коррекции,ΔX,ΔY,ΔZ,Xr + ΔX - X реал.,Yr + ΔY - Y реал.,Zr + ΔZ - Z реал.,Расстояние от реал до (Ur+Δ),Xr - ΔX - X реал.,Yr - ΔY - Y реал.,Zr - ΔZ - Z реал.,Расстояние от реал до (Ur-Δ)
0,218.0,4317.0,4928.0,0.094462,-0.028068,0.340452,-234.194188,681.058774,-275.057409,770.937449,-234.383112,681.114910,-275.738313,771.287612
1,451.0,3638.0,5202.0,0.102490,0.005967,-0.191805,-1.186160,2.092809,-1.589666,2.883377,-1.391140,2.080875,-1.206056,2.778468
2,451.0,3638.0,5202.0,0.226552,0.319607,-0.217655,-1.062098,2.406449,-1.615516,3.086899,-1.515202,1.767235,-1.180206,2.609950
3,453.0,3634.0,5204.0,-0.076623,-0.098018,-0.159191,0.634727,-2.011176,0.442948,2.154973,0.787973,-1.815140,0.761330,2.120202
4,452.0,3639.0,5202.0,0.067642,0.262147,-0.170503,-0.221008,3.348989,-1.568364,3.704637,-0.356292,2.824695,-1.227358,3.100364
5,451.0,3638.0,5202.0,0.150229,-0.482959,-0.041960,-1.138421,1.603883,-1.439821,2.437524,-1.438879,2.569801,-1.355901,3.242332
6,452.0,3640.0,5202.0,0.471494,-0.064819,0.389847,0.182844,4.022023,-1.008014,4.150445,-0.760144,4.151661,-1.787708,4.583667
7,451.0,3638.0,5202.0,0.119873,-0.204787,-0.129542,-1.168777,1.882055,-1.527403,2.690935,-1.408523,2.291629,-1.268319,2.973909
8,454.0,3630.0,5206.0,-0.147702,-0.270752,0.233624,1.563648,-6.183910,2.835763,6.980494,1.859052,-5.642406,2.368515,6.395520
9,454.0,3631.0,5205.0,0.044068,0.038563,-0.474408,1.755418,-4.874595,1.127731,5.302353,1.667282,-4.951721,2.076547,5.622403


In [94]:
results[2]

,Xr без коррекции,Yr без коррекции,Zr без коррекции,ΔX,ΔY,ΔZ,Xr + ΔX - X реал.,Yr + ΔY - Y реал.,Zr + ΔZ - Z реал.,Расстояние от реал до (Ur+Δ),Xr - ΔX - X реал.,Yr - ΔY - Y реал.,Zr - ΔZ - Z реал.,Расстояние от реал до (Ur-Δ)
0,224.769369,4296.937722,4935.774798,7.078886,-20.715115,8.368222,-220.440396,640.309449,-259.254840,725.122908,-234.598167,681.739679,-275.991285,771.995129
1,465.106320,3617.720289,5213.336914,14.509693,-20.694632,11.404941,27.327363,-38.887501,21.343994,52.101714,-1.692023,2.501764,-1.465889,3.357171
2,429.216328,3662.113522,5186.504776,-20.951680,23.771103,-15.245688,-44.024003,49.971467,-32.138773,73.947015,-2.120642,2.429260,-1.647396,3.621097
3,460.037114,3617.223237,5212.076231,6.865290,-16.635756,7.822655,14.613754,-35.325677,16.501025,41.638313,0.883174,-2.054165,0.855715,2.394126
4,450.701445,3646.418483,5199.244396,-1.223761,7.632631,-2.903887,-2.810966,18.137956,-7.057352,19.664516,-0.363445,2.872694,-1.249579,3.153714
5,440.903399,3656.212342,5193.445213,-9.783621,17.456541,-8.442456,-21.168872,37.755726,-18.395104,47.031860,-1.601630,2.842643,-1.510192,3.595347
6,453.257335,3634.078072,5204.128728,1.739010,-6.037880,2.541448,2.707695,-7.872966,3.272316,8.945572,-0.770325,4.202794,-1.810581,4.640591
7,460.891372,3623.275674,5210.087844,10.157470,-15.151857,8.090084,18.760192,-27.789340,14.780067,36.642088,-1.554748,2.514373,-1.400101,3.271024
8,452.165917,3635.253177,5203.572921,-2.001787,5.045337,-2.219335,-2.124519,4.385356,-2.044275,5.284315,1.879054,-5.705318,2.394396,6.466423
9,449.769484,3643.754882,5199.677062,-4.305969,13.162765,-5.948737,-6.825134,21.004488,-9.669535,24.109560,1.786803,-5.321041,2.227938,6.039027
